In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy
import random

from TrainingAlgorithm import TrainingAlgo
from DataPreprocess import create_dataloader
from model import TwoLayerNN

In [12]:
def All_r_LG_UA_w_LG_UA(model=None,
                        reg_params=None,
                        weight_params=None,
                        l2_lambda=0.001,
                        l1_lambda=0.001,
                        k=1,
                        p=50):
    
    prune_index = 0
    
    while not k>p:
        prune_index = random.randint(0, 127)
        
        train_algo.multiclass_regularization(epochs=reg_params['epochs'],
                                            model=model,
                                            optimizer=reg_params['optimizer'],
                                            loss_threshold=reg_params['loss_threshold'],
                                            eta_threshold=reg_params['eta_threshold'],
                                            l2_lambda=l2_lambda,
                                            l1_lambda=l1_lambda)
        
        saved_model = copy.deepcopy(model)
        
        model.del_neuron(index=prune_index)
        
        situation = train_algo.multiclass_weight_tuning(epochs=weight_params['epochs'],
                                                        model=model,
                                                        optimizer=weight_params['optimizer'],
                                                        loss_threshold=weight_params['loss_threshold'],
                                                        eta_threshold=weight_params['eta_threshold'])
        
        if situation == 'Unacceptable':
            print('All : Restore Network')
            model = saved_model
            print(model)
            k += 1
        
        elif situation == 'Acceptable':
            print('Pruning Works!')
            print(model)
            p -= 1
    
    print('finish training')
    return model

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TwoLayerNN(12, 128, 2, batch_norm=True, dropout=True)
train_loader, val_loader = create_dataloader()

# parameters for different task
reg_optimizer = optim.Adam(model.parameters(), lr=1e-3)
reg_loss_threshold = 0.6
reg_eta_threshold = 1e-6
reg_epochs = None

weight_optimizer = optim.Adam(model.parameters(), lr=1e-3)
weight_loss_threshold = 0.3
weight_eta_threshold = 1e-6
weight_epochs = 10

reg = {'optimizer': reg_optimizer,
       'loss_threshold': reg_loss_threshold,
       'eta_threshold': reg_eta_threshold,
       'epochs': reg_epochs}

weight = {'optimizer': weight_optimizer,
          'loss_threshold': weight_loss_threshold,
          'eta_threshold': weight_eta_threshold,
          'epochs': weight_epochs}

criterion = nn.CrossEntropyLoss()
train_algo = TrainingAlgo(train_loader, val_loader, criterion, device)
model = All_r_LG_UA_w_LG_UA(model, reg, weight, l2_lambda=0.001, l1_lambda=0.001, p=50)
print(model)

--------initializing regularization--------
max loss:0.9177491664886475 > threshold0.6, stop training.
[ 1 ] | train_loss = 0.67646, train_acc = 0.58073, val_loss = 0.43659, val_acc = 0.82292
--------initializing weight tuning--------
[ 1/10 ] | train_loss = 0.61967, train_acc = 0.67188, val_loss = 0.63233, val_acc = 0.73958
[ 2/10 ] | train_loss = 0.61833, train_acc = 0.67188, val_loss = 0.51566, val_acc = 0.77083
[ 3/10 ] | train_loss = 0.60966, train_acc = 0.66406, val_loss = 0.47327, val_acc = 0.75000
[ 4/10 ] | train_loss = 0.60754, train_acc = 0.70052, val_loss = 0.71947, val_acc = 0.75000
[ 5/10 ] | train_loss = 0.59417, train_acc = 0.68490, val_loss = 0.61813, val_acc = 0.75000
learning rate < threshold
[ 6/10 ] | train_loss = 0.62200, train_acc = 0.63021, val_loss = 0.67192, val_acc = 0.72917
All : Restore Network
TwoLayerNN(
  (layer_1): Linear(in_features=12, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p